## **Fraud Detection**

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns


Chargement du dataset

In [ ]:
df=pd.read_csv('/Users/hadjirariad/Desktop/Projet Github/creditcard.csv')

Aperçu

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Exploration des données

In [ ]:
print(df['Class'].value_counts(normalize=True))

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), cmap='coolwarm', linewidths=0.5)
plt.title("Matrice de corrélation")
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(df['Amount'], bins=50)
plt.title("Distribution des montants de transaction")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df['scaled_amount']= StandardScaler().fit_transform(df[['Amount']])
df['scaled_time']= StandardScaler().fit_transform(df[['Time']])

In [ ]:
df.drop(['Time','Amount'], axis=1, inplace=True)

In [ ]:
df = df[['scaled_time', 'scaled_amount'] + [col for col in df.columns if col.startswith('V')] + ['Class']]

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
X=df.drop('Class',axis=1)
y_true=df['Class']

In [ ]:
iso_forest=IsolationForest(contamination=0.001, random_state=42)
y_pred=iso_forest.fit_predict(X)

In [ ]:
y_pred = np.where(y_pred == -1,1,0)
print(classification_report(y_true, y_pred))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d')

In [ ]:
df['anomaly_pred']=y_pred
df['true_class']=y_true

In [ ]:
sns.countplot(x='anomaly_pred',hue='true_class',data=df)
plt.title("Comparaison anomalies détectées vs vraies fraudes")
plt.show()

Évaluation des perfs

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_score=roc_auc_score(y_true, y_pred)
print(f"AUC ROC Score: {roc_score:.4f}")

Comparaison avec un autre modèle

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
lof=LocalOutlierFactor(n_neighbors=20,contamination=0.001)
y_pred_lof=lof.fit_predict(X)
y_pred_lof=np.where(y_pred_lof == -1,1,0)

print(classification_report(y_true,y_pred_lof))